Criar uma tabela de ordens ja com:<br>
quantidade de itens, vendendores, lojas, mapeamento de Status e valor total da venda (valor total - desconto)<br><br>
Mapeamento Status:<br>
1 THEN 'Pending'<br>
2 THEN 'Processing'<br>
3 THEN 'Shipped'<br>
4 THEN 'Delivered'<br>
ELSE 'Unknown'<br>

In [0]:
%sql
USE CATALOG bikestore;

In [0]:
# Definir pastas do projetos em variaveis para facilitar
bronze_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/resource/origem'
resource_path_volume = '/Volumes/bikestore/logistics/bikestore_rescource/origem/'

In [0]:
# criar varias tabelas temporárias de forma prática
bronze_map = {
    "tmp_bronze_brands":      f"{bronze_path}/brands/",
    "tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_details/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    "tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    "tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%sql
select * from tmp_bronze_orders limit 10;

In [0]:
%sql
DESCRIBE  tmp_bronze_orders

In [0]:
%sql
SELECT 
  ORD.order_id,
  ORD.customer_id,
  ORD.order_status,
  ORD.order_date,
  ORD.required_date,
  ORD.shipped_date,
  ORD.store_id,
  ORD.staff_id,
  ST.store_name,
  ST.state,
  ST.city
FROM tmp_bronze_orders AS ORD
LEFT JOIN tmp_bronze_stores AS ST
  ON ORD.store_id = ST.store_id
LIMIT 5;

In [0]:
%sql
select * from tmp_bronze_stores;

In [0]:
%sql
DESCRIBE  tmp_bronze_brands;

In [0]:
%sql
SELECT * FROM tmp_bronze_staffs LIMIT 5;

In [0]:
%sql
DESCRIBE tmp_bronze_staffs

In [0]:
%sql
SELECT 
  ORD.order_id,
  ORD.customer_id,
  CASE
      WHEN ORD.order_status = 1 THEN 'Pending'
      WHEN ORD.order_status = 2 THEN 'Processing'
      WHEN ORD.order_status = 3 THEN 'Shipped'
      WHEN ORD.order_status = 4 THEN 'Delivered'
      WHEN ORD.order_status = 5 THEN 'Unknown'
  END AS order_status_desc,
  ORD.order_date,
  ORD.required_date,
  ORD.shipped_date,
  ST.store_name,
  ST.state,
  ST.city,
  CONCAT(STF.first_name, ' ', STF.last_name) AS full_name_staff,
  STF.email AS email_staff,
  STF.active AS active_staff
FROM tmp_bronze_orders AS ORD
LEFT JOIN tmp_bronze_stores AS ST
  ON ORD.store_id = ST.store_id
LEFT JOIN tmp_bronze_staffs AS STF
  ON ORD.staff_id = STF.staff_id;


In [0]:
%sql
select * from tmp_bronze_order_items;

In [0]:
%sql
DESCRIBE tmp_bronze_order_items;

In [0]:
%sql

WITH AS 
SELECT
  IO.order_id,
  IO.item_id,
  IO.product_id,
  IO.quantity,
  IO.list_price,
  IO.discount,
  round(IO.quantity * IO.list_price * (1 - IO.discount), 2) AS total_sale
FROM tmp_bronze_order_items AS IO;


In [0]:
%sql
SELECT
  order_id,
  item_id,
  product_id,
  quantity,
  list_price,
  discount,
  quantity * list_price * (1 - discount) AS total_after_discount
FROM tmp_bronze_order_items


In [0]:
%sql
WITH order_items AS (
  SELECT
    IO.order_id,
    IO.item_id,
    IO.product_id,
    IO.quantity,
    IO.list_price,
    IO.discount,
    round(IO.quantity * IO.list_price * (1 - IO.discount), 2) AS total_sale
  FROM tmp_bronze_order_items AS IO
)

SELECT 
  -- Orders
  ORD.order_id,
  ORD.customer_id,
  CASE
      WHEN ORD.order_status = 1 THEN 'Pending'
      WHEN ORD.order_status = 2 THEN 'Processing'
      WHEN ORD.order_status = 3 THEN 'Shipped'
      WHEN ORD.order_status = 4 THEN 'Delivered'
      WHEN ORD.order_status = 5 THEN 'Unknown'
  END AS order_status_desc,
  ORD.order_date,
  ORD.required_date,
  ORD.shipped_date,

  -- Store
  ST.store_name,
  ST.state,
  ST.city,

  -- Staff
  CONCAT(STF.first_name, ' ', STF.last_name) AS full_name_staff,
  STF.email AS email_staff,
  STF.active AS active_staff,

  -- Order Items (todas as colunas)
  OI.product_id,
  OI.quantity,
  OI.list_price,
  OI.discount,
  OI.total_sale

FROM tmp_bronze_orders AS ORD
LEFT JOIN tmp_bronze_stores AS ST
  ON ORD.store_id = ST.store_id
LEFT JOIN tmp_bronze_staffs AS STF
  ON ORD.staff_id = STF.staff_id
LEFT JOIN order_items AS OI
  ON ORD.order_id = OI.order_id;


In [0]:
df_orders_silver = spark.sql("""
    WITH order_items AS (
        SELECT
            IO.order_id,
            IO.item_id,
            IO.product_id,
            IO.quantity,
            IO.list_price,
            IO.discount,
            round(IO.quantity * IO.list_price * (1 - IO.discount), 2) AS total_sale
        FROM tmp_bronze_order_items AS IO
    )

    SELECT 
        -- Orders
        ORD.order_id,
        ORD.customer_id,
        CASE
            WHEN ORD.order_status = 1 THEN 'Pending'
            WHEN ORD.order_status = 2 THEN 'Processing'
            WHEN ORD.order_status = 3 THEN 'Shipped'
            WHEN ORD.order_status = 4 THEN 'Delivered'
            WHEN ORD.order_status = 5 THEN 'Unknown'
        END AS order_status_desc,
        ORD.order_date,
        ORD.required_date,
        ORD.shipped_date,

        -- Store
        ST.store_name,
        ST.state,
        ST.city,

        -- Staff
        CONCAT(STF.first_name, ' ', STF.last_name) AS full_name_staff,
        STF.email AS email_staff,
        STF.active AS active_staff,

        -- Order Items
        OI.product_id,
        OI.quantity,
        OI.list_price,
        OI.discount,
        OI.total_sale

    FROM tmp_bronze_orders AS ORD
    LEFT JOIN tmp_bronze_stores AS ST
        ON ORD.store_id = ST.store_id
    LEFT JOIN tmp_bronze_staffs AS STF
        ON ORD.staff_id = STF.staff_id
    LEFT JOIN order_items AS OI
        ON ORD.order_id = OI.order_id
""")

df_orders_silver.write \
    .mode("overwrite") \
    .format("delta") \
    .option("mergeSchema", "true") \
    .save(f"{silver_path}/orders_enriched/")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS bikestore.logistics.silver_orders
LOCATION "abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/orders_enriched"

In [0]:
%sql
SELECT * FROM bikestore.logistics.silver_orders;